# HPC Mini-Challenge 2 - Beschleunigung in Data Science
## Teil 2: GPU
#### FHNW - FS2024

Original von S. Suter, angepasst von S. Marcin und M. Stutz

Abgabe von: <font color='blue'>Name hier eintragen</font>

#### Ressourcen
* [Überblick GPU Programmierung](https://www.cherryservers.com/blog/introduction-to-gpu-programming-with-cuda-and-python)
* [CUDA Basic Parts](https://nyu-cds.github.io/python-gpu/02-cuda/)
* [Accelerate Code with CuPy](https://towardsdatascience.com/heres-how-to-use-cupy-to-make-numpy-700x-faster-4b920dda1f56)
* Vorlesungen und Beispiele aus dem Informatikkurs PAC (parallel computing), siehe Ordner "resources"
* CSCS "High-Performance Computing with Python" Kurs, Tag 3: 
    - JIT Numba GPU 1 + 2
    - https://youtu.be/E4REVbCVxNQ
    - https://github.com/eth-cscs/PythonHPC/tree/master/numba-cuda
    - Siehe auch aktuelles Tutorial von 2021
* [Google CoLab](https://colab.research.google.com/) oder ggf. eigene GPU.


In [ ]:
#!pip install numba

DIS ISSUE: https://github.com/numba/numba/issues/7104

NUMBA_CUDA_DRIVER="/usr/lib/wsl/lib/libcuda.so.1" python -c "from numba import cuda; cuda.detect()" -> this works

In [1]:
!numba -s

System info:
/home/buesst1/.cache/pypoetry/virtualenvs/hpc-mc2-buesst1-bcMvFOTC-py3.11/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
--------------------------------------------------------------------------------
__Time Stamp__
Report started (local time)                   : 2024-11-18 20:32:45.172829
UTC start time                                : 2024-11-18 19:32:45.172831
Running time (s)                              : 0.566623

__Hardware Information__
Machine                                       : x86_64
CPU Name                                      : goldmont
CPU Count                                     : 32
Number of accessible CPUs     

In [1]:
# Dummy Beispiel zum testen mit Numba

import math
from numba import vectorize
import numpy as np

@vectorize(['float32(float32)'], target='cuda')
def gpu_sqrt(x):
    return math.sqrt(x)
  
a = np.arange(4096,dtype=np.float32)
gpu_sqrt(a)

/home/buesst1/.cache/pypoetry/virtualenvs/hpc-mc2-buesst1-bcMvFOTC-py3.11/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 6 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([ 0.       ,  1.       ,  1.4142135, ..., 63.97656  , 63.98437  ,
       63.992188 ], dtype=float32)

### 5 GPU Rekonstruktion

Implementiere eine SVD-Rekonstruktionsvariante auf der GPU oder in einem hybriden Setting. Code aus dem ersten Teil darf dabei verwendet werden. Wähle  bewusst, welche Teile des Algorithms in einem GPU Kernel implementiert werden und welche effizienter auf der CPU sind. Ziehe dafür Erkenntnisse aus dem ersten Teil mit ein. Es muss mindestens eine Komponente des Algorithmuses in einem GPU-Kernel implementiert werden. Dokumentiere Annahmen, welche du ggf. zur Vereinfachung triffst. Evaluiere, ob du mit CuPy oder Numba arbeiten möchtest.

Links:
* [Examples: Matrix Multiplikation](https://numba.readthedocs.io/en/latest/cuda/examples.html)

In [25]:
import numpy as np
from numba import cuda
import time

class time_region:
    def __init__(self, time_offset=0):
        self._time_offset = time_offset

    def __enter__(self):
        self._t_start = time.time()
        return self
    
    def __exit__(self, exec_type, exec_value, traceback):
        self._t_end = time.time()

    def elapsed_time(self):
        return self._time_offset + (self._t_end - self._t_start)

class time_region_cuda:
    def __init__(self, time_offset=0, cuda_stream=0):
        self._t_start = cuda.event(timing=True)
        self._t_end = cuda.event(timing=True)
        self._time_offset = time_offset
        self._cuda_stream = cuda_stream
    
    def __enter__(self):
        self._t_start.record(self._cuda_stream)
        return self

    def __exit__(self, exec_type, exec_value, traceback):
        self._t_end.record(self._cuda_stream)
        self._t_end.synchronize()

    def elapsed_time(self):
        return self._time_offset + 1e-3*cuda.event_elapsed_time(self._t_start, self._t_end)

@cuda.jit("void(Array(float32, 2, 'C'), Array(float32, 1, 'C'), Array(float32, 2, 'F'), int32, Array(float32, 2, 'C'))")
def svd_reco_kernel(u, s, vt, k, y):
    """SVD reconstruction for k components using cuda
    
    Inputs:
    u (m,n): array
    s (n): array (diagonal matrix)
    vt (n,n): array
    k int: number of reconstructed singular components
    y (m,n): output array
    """
    m, n = cuda.grid(2)

    if m >= u.shape[0] or n >= vt.shape[0]:
        return

    element = 0.0
    for p in range(k):
        element += u[m, p] * s[p] * vt[p, n]

    y[m, n] = element

def calculate(u: np.ndarray, s: np.ndarray, vt: np.ndarray, k: int, block_size:tuple=(32,32)) -> np.ndarray:
    """Host function to perform SVD reconstruction using CUDA kernel.

    Args:
        u (np.ndarray): Left singular vectors, shape (m, n).
        s (np.ndarray): Singular values, shape (n,).
        vt (np.ndarray): Right singular vectors, shape (n, n).
        k (int): Number of singular components to use in reconstruction.
        block_size (tuple(int,int)): Number of threads per block

    Returns:
        np.ndarray: Reconstructed matrix, shape (m, n).
    """
    
    # convert to correct dtype
    u = u.astype(np.float32)
    s = s.astype(np.float32)
    vt = np.asfortranarray(vt.astype(np.float32))

    with time_region_cuda() as t_xfer:
        # Ensure inputs are in the correct dtype and order
        u = cuda.to_device(u)
        s = cuda.to_device(s)
        vt = cuda.to_device(vt)
        
        # create array where results are stored. Also pin that array so no data gets moved out of the ram.
        m, n = u.shape[0], vt.shape[1]
        y = cuda.device_array((m, n), dtype=np.float32)
        y_ret = cuda.pinned_array_like(y)

    # Define CUDA thread and block dimensions
    blocks_per_grid_x = (m + block_size[0] - 1) // block_size[0]
    blocks_per_grid_y = (n + block_size[1] - 1) // block_size[1]
    blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

    with time_region_cuda() as t_kernel:
        # Launch the CUDA kernel
        svd_reco_kernel[blocks_per_grid, block_size](u, s, vt, k, y)

    with time_region_cuda(t_xfer.elapsed_time()) as t_xfer:
        # copy back to host
        y.copy_to_host(y_ret)

    # calculate number of transfers done in each thread
    num_transfers_per_thread = 1 + (4*k) + 1
    number_of_GB_transferred = 1e-9*4*num_transfers_per_thread*y.shape[0]*y.shape[1]

    print(f"Cuda transfer overhead: {t_xfer.elapsed_time()*1000}ms")
    print(f"Cuda kernel time: {t_kernel.elapsed_time()*1000}ms")
    print(f"Consumed memory bandwidth: {number_of_GB_transferred / t_kernel.elapsed_time()} GB/s")

    return y_ret


In [36]:
import os
import imageio.v3 as imageio
import numpy as np
import glob

subfolder = '001'
folders = os.path.join('adni_png', subfolder)

# Get all PNGs from 001 with 145 in the name
files = sorted(glob.glob(f"{folders}/*145.png"))

# Load all images using ImageIO and create a numpy array from them
images = np.array([imageio.imread(f) for f in files])

# Get all the names of the files
names = [f[-17:-4] for f in files]

im = images[0]
im = im - im.min() / im.max() - im.min()  # normalize image
u, s, vt = np.linalg.svd(im, full_matrices=False)

# convert to correct dtype
u = u.astype(np.float32)
s = s.astype(np.float32)
vt = vt.astype(np.float32)

calculate(u, s, vt, 100)

Cuda transfer overhead: 1.002495974302292ms
Cuda kernel time: 0.5836799740791321ms
Consumed memory bandwidth: 119.89474216655664 GB/s


array([[ 0.04974082,  0.00398783,  0.06063343, ...,  0.06974227,
         0.06019334, -0.04235162],
       [ 0.02688764,  0.00190527,  0.02418954, ...,  0.03627283,
         0.01780313, -0.00637781],
       [ 0.0290446 , -0.02647049, -0.03107073, ...,  0.04624368,
         0.00306286,  0.03151328],
       ...,
       [ 1.3522642 , -0.00273046, -0.623177  , ..., -0.6813155 ,
        -0.4641527 , -0.96072966],
       [ 0.2960358 ,  0.5249512 ,  0.0184323 , ..., -0.19332825,
         0.5000806 ,  1.5635303 ],
       [ 0.9867547 , -0.9557681 ,  1.4895947 , ...,  1.0953    ,
        -0.5926281 ,  1.8346974 ]], dtype=float32)

In [11]:
def reconstruct_svd_for_loops3(u,s,vt,k):
    """SVD reconstruction for k components using 3 for-loops
    
    Inputs:
    u: (m,n) numpy array
    s: (n) numpy array (diagonal matrix)
    vt: (n,n) numpy array
    k: number of reconstructed singular components
    
    Ouput:
    (m,n) numpy array U_mk * S_k * V^T_nk for k reconstructed components
    """
    ### BEGIN SOLUTION

    if k is None:
        k = min(u.shape[0], vt.shape[0])

    reco = np.zeros((u.shape[0], vt.shape[0]))

    for m in range(u.shape[0]):
        for n in range(vt.shape[0]):
            element = 0
            for p in range(k):
                element += u[m, p]*s[p]*vt[p, n]

            reco[m, n] = element
            
    ### END SOLUTION

    return reco

reconstruct_svd_for_loops3(u, s, vt, 20)

array([[ 9.48357570e-04,  6.87303161e-03,  5.32832742e-03, ...,
         8.10194574e-03,  5.62756276e-03,  1.05523104e-02],
       [ 5.10228332e-04, -5.43693895e-04,  8.77237471e-05, ...,
         2.47143582e-03, -7.51565312e-05,  2.96106702e-03],
       [ 3.63633828e-03,  3.92481964e-03,  1.46562816e-03, ...,
         7.42339715e-03,  2.51593534e-03,  1.07109863e-02],
       ...,
       [ 1.30805051e+00,  1.37908542e+00,  1.05979729e+00, ...,
         6.39316261e-01,  1.12439133e-01,  8.58197808e-02],
       [ 1.23633599e+00,  1.39993715e+00,  1.02188718e+00, ...,
         6.65664971e-01,  9.83572006e-02,  2.19427586e-01],
       [ 1.16466701e+00,  1.40309119e+00,  9.09437180e-01, ...,
         3.75059962e-01,  1.99167326e-01,  4.85913455e-02]])

<font color='blue'>Antwort hier eingeben</font>

#### 5.2 GPU-Kernel Performance

##### 5.3.1 Blocks und Input-Grösse

Links: 
* [Examples: Matrix Multiplikation](https://numba.readthedocs.io/en/latest/cuda/examples.html)
* [NVIDIA Kapitel zu "Strided Access"](https://spaces.technik.fhnw.ch/multimediathek/file/cuda-best-practices-in-c)
* https://developer.nvidia.com/blog/cublas-strided-batched-matrix-multiply/
* https://developer.nvidia.com/blog/how-access-global-memory-efficiently-cuda-c-kernels/

Führe 2-3 Experimente mit unterschiedlichen Blockkonfigurationen und Grösse der Input-Daten durch. Erstelle dafür ein neues Datenset mit beliebig grossen Matrizen, da die GPU besonders geeignet ist um grosse Inputs zu verarbeiten (Verwende diese untschiedlich grossen Matrizen für alle nachfolgenden Vergeliche und Tasks ebenfalls). Messe die Performance des GPU-Kernels mittels geeigneten Funktionen. Welche Blockgrösse in Abhängigkeit mit der Input-Grösse hat sich bei dir basierend auf deinen Experimenten als am erfolgreichsten erwiesen? Welches sind deiner Meinung nach die Gründe dafür? Wie sind die Performance Unterschiede zwischen deiner CPU und GPU Implementierung? Diskutiere deine Analyse (ggf. mit Grafiken).

In [ ]:
### BEGIN SOLUTION




### END SOLUTION

<font color='blue'>Antwort hier eingeben</font>

##### 5.2.2 Shared Memory auf der GPU
Optimiere deine Implementierung von oben indem du das shared Memory der GPU verwendest. Führe wieder mehrere Experimente mit unterschiedlicher Datengrösse durch und evaluiere den Speedup gegenüber der CPU Implementierung.

Links:
* [Best Practices Memory Optimizations](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#memory-optimizations)
* [Examples: Matrix Multiplikation und Shared Memory](https://numba.readthedocs.io/en/latest/cuda/examples.html)

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

Was sind deine Erkenntnisse bzgl. GPU-Memory-Allokation und des Daten-Transferes auf die GPU? Interpretiere deine Resultate.

<font color='blue'>Antwort hier eingeben</font>

##### 5.2.3 Bonus: Weitere Optimierungen
Optimiere deine Implementation von oben weiter. Damit du Erfolg hast, muss der Data-Reuse noch grösser sein.

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

#### 5.3 NVIDIA Profiler

Benutze einen Performance Profiler von NVIDIA, um Bottlenecks in deinem Code zu identifizieren bzw. unterschiedliche Implementierungen (Blocks, Memory etc.) zu vergleichen. 

* Siehe Beispiel example_profiling_CUDA.ipynb
* [Nsight](https://developer.nvidia.com/nsight-visual-studio-edition) für das Profiling des Codes und die Inspektion der Ergebnisse (neuste Variante)
* [nvprof](https://docs.nvidia.com/cuda/profiler-users-guide/index.html#nvprof-overview)
* [Nvidia Visual Profiler](https://docs.nvidia.com/cuda/profiler-users-guide/index.html#visual)

> Du kannst NVIDIA Nsights Systems und den Nvidia Visual Profiler auf deinem PC installieren und die Leistungsergebnisse aus einer Remote-Instanz visualisieren, auch wenn du keine GPU an/in deinem PC hast. Dafür kannst du die ``*.qdrep`` Datei generieren und danach lokal laden.


Dokumentiere deine Analyse ggf. mit 1-2 Visualisierungen und beschreibe, welche Bottlenecks du gefunden bzw. entschärft hast.

<font color='blue'>Antwort hier eingeben inkl. Bild.</font>

### 6 Beschleunigte Rekonstruktion mehrerer Bilder
#### 6.1 Implementierung
Verwende einige der in bisher gelernten Konzepte, um mehrere Bilder gleichzeitig parallel zu rekonstruieren. Weshalb hast du welche Konzepte für deine Implementierung verwenden? Versuche die GPU konstant auszulasten und so auch die verschiedenen Engines der GPU parallel zu brauchen. Untersuche dies auch für grössere Inputs als die MRI-Bilder.

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

<font color='blue'>Antwort hier eingeben</font>

#### 6.2 Analyse
Vergleiche den Speedup für deine parallele Implementierung im Vergleich zur seriellen Rekonstruktion einzelner Bilder. Analysiere und diskutiere in diesem Zusammenhang die Gesetze von Amdahl und Gustafson.

<font color='blue'>Antwort hier eingeben</font>

#### 6.3 Komponentendiagramm

Erstelle das Komponentendiagramm dieser Mini-Challenge für die Rekunstruktion mehrere Bilder mit einer GPU-Implementierung. Erläutere das Komponentendigramm in 3-4 Sätzen.


<font color='blue'>Antwort hier eingeben inkl. Bild(ern).</font>

### 7 Reflexion

Reflektiere die folgenden Themen indem du in 3-5 Sätzen begründest und anhand von Beispielen erklärst.

1: Was sind deiner Meinung nach die 3 wichtigsten Prinzipien bei der Beschleunigung von Code?

<font color='blue'>Antwort hier eingeben</font>

2: Welche Rechenarchitekturen der Flynnschen Taxonomie wurden in dieser Mini-Challenge wie verwendet?

<font color='blue'>Antwort hier eingeben</font>

3: Haben wir es in dieser Mini-Challenge hauptsächlich mit CPU- oder IO-Bound Problemen zu tun? Nenne Beispiele.

<font color='blue'>Antwort hier eingeben</font>

4: Wie könnte diese Anwendung in einem Producer-Consumer Design konzipiert werden?

<font color='blue'>Antwort hier eingeben</font>

5: Was sind die wichtigsten Grundlagen, um mehr Performance auf der GPU in dieser Mini-Challenge zu erreichen?

<font color='blue'>Antwort hier eingeben</font>

6: Reflektiere die Mini-Challenge. Was ist gut gelaufen? Wo gab es Probleme? Wo hast du mehr Zeit als geplant gebraucht? Was hast du dabei gelernt? Was hat dich überrascht? Was hättest du zusätzlich lernen wollen? Würdest du gewisse Fragestellungen anders formulieren? Wenn ja, wie?

<font color='blue'>Antwort hier eingeben</font>